In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
from numpy.random import default_rng
sys.path.append('../utilities')
import plotting_utils as utils
import simulation_utils as sim
from scipy.optimize import minimize


In [ ]:
bounds = {"radius": [90, 265,5], "thickness": [5,20,1], "npanels": [4,360,4], "theta": [0,90,2], "length": [2,150,2] }
number_of_samples = 300
num_digits = 2
version='v1.4'

height=300
zpos=42
design=4


In [ ]:
def get_random_samples(bounds, num_samples=9, seed=None):
    rng = default_rng(seed)
    samples = {}
    for parameter, bound in bounds.items():
        samples[parameter] = rng.uniform(bound[0], bound[1], num_samples)
    return pd.DataFrame(samples)

In [ ]:
def get_latin_hypercube_samples(bounds, num_samples=10):
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    design_space = DesignSpace(
            [
                FloatVariable(l_bounds[0], u_bounds[0]),
                FloatVariable(l_bounds[2],u_bounds[2]),
                IntegerVariable(l_bounds[1],u_bounds[1]),
                FloatVariable(l_bounds[3],u_bounds[3]),
                FloatVariable(l_bounds[4], u_bounds[4]),
            ]
        )
    sampler = MixedIntegerSamplingMethod(LHS, design_space, criterion="ese", random_state=42)
    samples = sampler(num_samples)
    return pd.DataFrame(samples, columns=list(bounds.keys()))

def get_latin_hypercube_samples_extended(samples, bounds, num_samples_to_add=10):
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    design_space = DesignSpace(
            [
                FloatVariable(l_bounds[0], u_bounds[0]),
                FloatVariable(l_bounds[2],u_bounds[1]),
                IntegerVariable(l_bounds[1],u_bounds[2]),
                FloatVariable(l_bounds[3],u_bounds[3]),
                FloatVariable(l_bounds[4], u_bounds[4]),
            ]
        )
    sampler = MixedIntegerSamplingMethod(LHS, design_space, criterion="ese", random_state=42)
    samples_continued = sampler.expand_lhs(samples, num_samples_to_add, method="ese")
    return pd.DataFrame(samples_continued, columns=list(bounds.keys()))

In [ ]:
samples = []
n=0
while n < 300:
    s = get_random_samples(bounds, 1,seed=None).to_numpy()[0]
    if get_inner_radius(s) < 90.:
        continue
    elif get_outer_radius(s) > 265.:
        continue
    elif get_outer_radius(s)-get_inner_radius(s)  > 20.:
        continue
    elif s[2]*s[1]*s[4] > np.pi*(get_outer_radius(s)**2-get_inner_radius(s)**2):
            continue
    else:
        samples.append(s)
        n = len(samples)
    
df=pd.DataFrame(samples,columns=bounds.keys())
print(df)

In [ ]:
labels = ["radius",  "thickness",  "npanels",  "theta",  "length"]
fig, axs = plt.subplots(len(labels), len(labels),figsize=(9,9), layout="constrained")
for i in range(len(labels)):
    for j in range(len(labels)):
        axs[j,i].scatter(df[labels[i]].to_numpy(),df[labels[j]].to_numpy(),s=5)
        if i==0:
            axs[j,i].set_ylabel(labels[j])
        if j==len(labels)-1:
            axs[j,i].set_xlabel(labels[i])

In [ ]:
for s in samples:
    utils.draw_moderator_configuration(s)
    plt.show()

In [2]:
if os.path.exists(f'../simulation/LF/{version}/macros')==False:
   os.makedirs(f'../simulation/LF/{version}/macros')
for sample in samples:  
   sim.print_geant4_macro(sample, version, mode='LF')

NameError: name 'version' is not defined

In [ ]:
if os.path.exists(f'./out/{version}')==False:
   os.makedirs(f'out/{version}')
df.to_csv(f'out/{version}/LF-sim-samples_{version}.csv')